In [1]:
!pip install transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-hate')

# Load the pre-trained RoBERTa model for hate speech detection
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-hate')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

df = pd.read_csv('test_hate_dataset.csv')

# Create binary labels: 0 for non-hate speech (offensive and neither), 1 for hate speech
# DA CAMBIARE
df['label'] = df['label'].apply(lambda x: 1 if x == 0 else 0)
test_texts= df['text']
test_labels=df['label']
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-hate')

# Load the pre-trained RoBERTa model for hate speech detection
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-hate')

# Define a function to classify text using the model
def classify_hate_speech(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    return predicted_class

# Define a function to evaluate the model
def evaluate_model(test_texts, test_labels):
    predictions = []
    for text in tqdm(test_texts, desc="Classifying texts"):
        predicted_class = classify_hate_speech(text)
        predictions.append(predicted_class)
    return predictions

# Get predictions for the test set
predictions = evaluate_model(test_texts, test_labels)
print(len(predictions))
# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f'Accuracy: {accuracy:.4f}')

# Salva il DataFrame in un file CSV

# Print detailed classification report
print(classification_report(test_labels, predictions, target_names=['Non-Hate Speech', 'Hate Speech']))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Classifying texts: 100%|██████████| 16458/16458 [47:38<00:00,  5.76it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


16458
Accuracy: 0.8565
                 precision    recall  f1-score   support

Non-Hate Speech       1.00      0.86      0.92     16458
    Hate Speech       0.00      0.00      0.00         0

       accuracy                           0.86     16458
      macro avg       0.50      0.43      0.46     16458
   weighted avg       1.00      0.86      0.92     16458



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
df = pd.read_csv('test_hate_dataset.csv')

In [20]:
df['prediction'] = predictions
df.head()

,text,label,prediction
0,social services have a love/hate relationship ...,nothate,0
1,Yes it's a joke and offensive isn't it? Why ar...,nothate,0
2,"I do not have passions in life, and I'm not su...",nothate,0
3,You better be. That was the most half-assed co...,nothate,0
4,"Waging war on, and bombing, a foreign sovereig...",hate,0


In [ ]:
df.to_csv('classified_sentences.csv', index=False)